Libraries import

In [348]:
from collections import namedtuple
from functools import partial
from random import choices, randint, randrange, random
from typing import List, Optional, Callable, Tuple
import random

Problem function to generate subsets + generation of the essential one

In [354]:
import random
random.seed(42)
#100 - 21 -> N-50
#500 - 17 -> n-250 // 500 - 57 -> N-400 // 500 - 105 -> N-450 (results 500.txt) // 500 - 255 -> N-475
#1000 - 38 -> N-700 // 1000 - 95 -> N-850 (result: 1000.txt)
N=500
num_of_subsets=105
#Generation of the universe
universe = [i for i in range(1,N+1)]

#generation of subsets
subsets = [set(sorted(random.sample(universe, random.randint(0, N-450)))) for i in range(num_of_subsets)]

#generation of weight - unitary for all of them
C = [random.randint(1, 1) for i in range(len(subsets))] #each subset weights 1

'''
Theoretically each set should have a weight
'''

#printing of input
print("universe:", universe)
print("\nsubsets:")
for sub in subsets:
    print(sorted(sub))
print("\nC", C)

#checking existence of cover
print("\nExistence of cover:")
elements = set(e for s in subsets for e in s)
if sorted(elements) != universe:
    print("No")
else:
    print("Yes")

universe: [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 22

**Genetic Algorithm**

For a GA we need 
- genetic representation of a solution
- function to generate new solutions
- fitness function
- selection function
- crossover function
- mutation function
To better understand which every of these means, please check the repo 'genetic_algorithm_from_scratch' in my github.


In [357]:
def genetic_algorithm(universe, subset, weights, N, max_gen=100):
    ''' 
    universe: set to be covered
    subsets: subsets
    weights: vector or weights (all 1)
    N: number of individuals
    max_gen: maximum number of eras
    '''
    #defining the fitness function:
    fitness = lambda ind: sum([C[j] * ind[j] for j in range(len(ind))])
    # Function of checking that individual is a cover
    #pretty much the same thing did previously but i need this function to check
    #if the final result has or not the cover
    def check(x):
        check_list = []
        for j in range(len(subsets)):
            if x[j]:
                check_list.append(subsets[j])
        # Checking existence of cover
        elem = set(e for s in check_list for e in s)
        if sorted(elem) != sorted(universe):
            return False
        else:
            return True
    #("STEP 1 --- generating the first population\n")
    #to understand why is all zeros and ones watch the repo "genetic_algorithm_from_scratch"
    FP=[[random.randint(0, 1) for j in range(len(subsets))] for i in range(N)]
    #I will use two variables: number_of_generations (nog) and changes (how many generations without changes)
    nog=0
    changes=0
    
    #I will do the following till nog=max_gen 
    while nog != max_gen:
        #i copy the "older" population 
        #("Saving the population...")
        past_population=FP.copy() 
        #("STEP 2 --- selection of two individuals by tournament selection\n")
        fitness_list=[] #all the fitness values
        for i in range(len(FP)):
            if sum(FP[i])==0: #basically no solution
                FP[i]=[1 for j in range(len(subsets))]
        for fp in FP:
            fitness_list.append(1/fitness(fp)) #calculating the fitness value of the subset
        
        summed_fitness_list=sum(fitness_list)
        #("Choosing parents\n")
        rand=random.random()

        #choosing parents
        #solutions with the highest value should be selected
        #i use a random value, otherwise i could sort
        for k in range(len(fitness_list)):
            if fitness_list[k]>rand:
                parent1=FP[k-1]
                idx1=k-1
                parent2=FP[k]
                idx2=k
                rand
                break
        #("STEP 3 --- Crossover\n")
        new_gen=[]
        p0 = sum([item for sublist in FP for item in sublist]) / (len(subsets) * N)
        p1 = 1 - p0
        ''' 
        This may seems difficult but I swear it is not:
        for j in range(len(parent1)) -> moving along parent length. Technically the two length
        should be equal.

        Here I basically do a tournament between the value of the parent. If it is 0, then I check the probability to insert 0;
        otherwise if the value is 1, i check the probability of 1. 
        p0 and p1 are the relatives probability, the biggest wins the tournament. 
        '''
        for j in range(len(parent1)):
            if parent1[j]==parent2[j]:
                new_gen.append(parent1[j])
            else:
                if parent1[j]==0:
                    if 1/fitness_list[idx1]*p0>=1/fitness_list[idx2]*p1:
                        new_gen.append(parent1[j])
                    else:
                        new_gen.append(parent2[j])
                else:
                    if 1/fitness_list[idx1]*p1>=1/fitness_list[idx2]*p0:
                        new_gen.append(parent1[j])
                    else:
                        new_gen.append(parent2[j])
        #("STEP 4 --- Mutation\n")
        #takes a genome n with a certain probability, change 1 to 0 and 0 to 
        H=[] #probability vector
        #("Mutating the new generation...\n")
        for j in range(len(new_gen)):
            H.append(random.uniform(0,2 / N)) #random 0 or 1
        for j in range(len(new_gen)):
            if H[j]>1/N and new_gen[j]==1 and p1>p0:
                #1 to 0
                new_gen[j]=0
            elif H[j]>1/N and new_gen[j]==1 and p0>p1:
                new_gen[j]=1 #change 0 to 1
        
        #("STEP 5 --- Run the algorithm\n")
        ''' 
        R -> set of all, included the element covered
        M -> all uncovered elements
        w -> number of subsets that contains the element
        '''
        M=0
        while M != set():
            #5.1 -- fill M
            K=[]
            #searching for the element in the universe that are in the single subset
            for elem in universe:
                var_list=set()
                var=0
                for subset in subsets:
                    if elem in subset:
                        var_list.add(var)
                    var+=1
                K.append(var_list)
            R = set([i for i in range(len(new_gen)) if new_gen[i]])
            W = [len(R & i) for i in K]
            M = set([list(universe)[i] for i in range(len(W)) if W[i] == 0])
            ''' 
            K -> contains the element that we find in the subset AND that belong to the universe too
            var -> how many times
            var_list -> list of var
            '''
            # Step 5.2
            var1=float("inf")
            var2=float("inf")
            S = None
            min_S = float("inf")
            for j in range(len(subsets)):
                if len(subsets[j] & M) != 0:
                    var1 = C[j] / len(subsets[j] & M)
                if var1 < min_S:
                    min_S = var1
                    S = subsets[j]
                    var2 = j
            if var2 !=float("inf"):
                R.add(var2)
            if S!=None:
                for i in range(len(W)):
                    if universe[i] in S:
                        W[i] += 1
                M = M - S
            
            # Step 5.3
            R_new = R.copy()
            for j in R:
                w_list = [W[universe.index(el)] > 1 for el in subsets[j]]
                if all(w_list):
                    R_new -= {j}
                    for i in range(len(W)):
                        if universe[i] in subsets[j]:
                            W[i] -= 1
            R = R_new
            M = set([list(universe)[i] for i in range(len(W)) if W[i] == 0])

            # Step 5.4 
            new_gen = [0] * len(subsets)
            for el in R:
                new_gen[el] = 1
        #("STEP 6 --- Replacing an individual in a population")
        if new_gen not in FP:
            d = [1 / x for x in fitness_list]
            aver = sum(d) / len(d)
            id_to_change = [i for i in range(len(d)) if d[i] >= aver]
            FP[random.choice(id_to_change)] = new_gen
        
        #Preparing for a new era
        nog += 1
        if past_population == FP:
            changes+=1
        else:
            changes=0
    # Verification that an individual in population X is a cover. If not, replace f with infinity
    res_list = []
    for x in FP:
        if check(x):
            res_list.append(fitness(x))
        else:
            res_list.append(float("inf"))
    
    min_id = res_list.index(min(res_list))
    res = FP[min_id]
    
    return nog,res_list[min_id],res,changes

Actual evolution

In [369]:
gen_res = genetic_algorithm(universe,subsets,C,N)
print("N:",N," \n")
#printing results 
print("iterations:",gen_res[0])
print("\nmin price: ",gen_res[1])
print("\nweight:\n",C)
print("\nresult individual :\n",gen_res[2])
print("\nset to cover:\n",universe)
print("\nsubsets to cover:")
for j in range(len(subsets)):
    if gen_res[2][j]:
        print(sorted(subsets[j]))

UnboundLocalError: local variable 'parent1' referenced before assignment